In [1]:
import sys
sys.path.append('./scripts')
sys.path.append('./probvlm')

from tuning_tools import prep_str_args, evaluate_checkpoint, get_evaluation_history, gridsearch, create_base_str_args

[nltk_data] Downloading package punkt to C:\Users\Laptop of
[nltk_data]     Natalie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Laptop of
[nltk_data]     Natalie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
history_path = './results/test_eval_ilt.txt'  
model_history = get_evaluation_history(history_path) # Get the history of evaluated models
eval_file = './results/eval_ilt.txt'
split = 'test'

base_str_args = create_base_str_args('baseline', 'ILT', eval_file)
print('base str args:', base_str_args)

# Dictionary of values to gridsearch for hyperparam tuning
gridsearch_dict = {
    '--epochs' : [5], #list(range(15,36,5)) if 'active-learning' in base_str_args else [35], #[10,15,20,25,30,35],
    '--batch-size' : [64],
    '--al-iter': [5], #list(range(3,17,2)), #list(range(1,6,2)),
    '--al-epochs': [10],
    '--label-ratio': [0.05, 0.1, 0.2, 0.4, 0.8],
    '--lr': [5e-5],
    '--test-data': ['ILT'],
}


Number of configs: 5


In [3]:
gridsearch(base_str_args, gridsearch_dict, model_history, eval_file, split, change_seed = True)

['--train-data', 'ILT', '--test-data', 'ILT', '--keyword-path', 'keywords/RS/class-name.txt', '--zeroshot-frequency', '5', '--method', 'base', '--lr', '5e-5', '--eval-file', './results/eval_ilt.txt', '--epochs', '5', '--batch-size', '64', '--al-iter', '5', '--al-epochs', '10', '--label-ratio', '0.05']
Config number 0: 5 repeats


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.62it/s]


KeyboardInterrupt: 